## Imports

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import scipy.io as sio
from characterDefinitions import getHandwritingCharacterDefinitions
from torchvision.models import resnet50
import numpy as np

In [7]:
data = sio.loadmat('./Datasets/t5.2019.05.08/singleLetters.mat')

## Load the Dataset

In [8]:
topDirs = ['Datasets']
dataDirs = ['t5.2019.05.08','t5.2019.11.25','t5.2019.12.09','t5.2019.12.11','t5.2019.12.18',
            't5.2019.12.20','t5.2020.01.06','t5.2020.01.08','t5.2020.01.13','t5.2020.01.15']
charDef = getHandwritingCharacterDefinitions()

In [27]:
all_tensors = []
all_labels = []
for directory in dataDirs:
    
    mat = f'./Datasets/{directory}/singleLetters.mat'
    data = sio.loadmat(mat)
    ctr = 0
    for letter in charDef['charList']:
        t = torch.Tensor(data[f'neuralActivityCube_{letter}'])
        qty = t.shape[0]
        labels = torch.Tensor([ctr]*qty)
        ctr += 1
#         if t.shape[0] == 27:
        all_tensors.append(t)
        all_labels.append(labels)

tensor_data = torch.cat(all_tensors, dim=0)
tensor_data = np.repeat(tensor_data[..., np.newaxis], 3, -1).transpose(-1,-2).transpose(-2,-3)

# tensor_data = tensor_data.transpose(-1,0).transpose(-1,-2)
tensor_labels = torch.cat(all_labels).long()

all_data=[]
def create_data_loaders():
    for k in range(31):
        currentLetterLabels =[]
        #re-label all not 0 labels to 1.
        for label,i in zip(tensor_labels,range(len(tensor_labels))):
            if label !=k:
                currentLetterLabels.append(1)
            else:
                currentLetterLabels.append(0)
        #take 117 samples that represent a with their samples.
        letter_data = tensor_data[117*k:117*(k+1)]
        letter_labels = torch.Tensor(currentLetterLabels[117*k:117*(k+1)])
        #take the rest that don't represent a with their samples.
        not_letter_data=[]
        if len(tensor_data[:117*k])!=0:
            not_letter_data.append(tensor_data[:117*k])
        if len(tensor_data[117*(k+1):])!=0:
            not_letter_data.append(tensor_data[117*(k+1):])
        not_letter_data = torch.cat(not_letter_data,dim=0)

        not_letter_labels=[]
        if len(currentLetterLabels[:117*k])!=0:
            not_letter_labels.append(torch.Tensor(currentLetterLabels[:117*k]))
        if len(currentLetterLabels[117*(k+1):])!=0:
            not_letter_labels.append(torch.Tensor(currentLetterLabels[117*(k+1):]))
        not_letter_labels = torch.cat(not_letter_labels,dim=0)
        

        #generate a random permutation and shuffle both labels and predictions 
        indices = torch.randperm(not_letter_data.size()[0])
        not_letter_data=not_letter_data[indices]
        not_letter_labels=not_letter_labels[indices]

        #take the first 117 from the shuffled arrays
        not_letter_data = not_letter_data[:117]
        not_letter_labels = not_letter_labels[:117]

        #add both the arrays in order to create a 50:50 dataset
        all_data.append((torch.cat([letter_data,not_letter_data],dim=0),torch.cat([letter_labels,not_letter_labels],dim=0).long()))
    data_loaders = []
    for i in range(31):
        dataset = TensorDataset(all_data[i][0], all_data[i][1])
        train_data, test_data = random_split(dataset, [187, 47])
        batch_size = 16
        train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
        data_loaders.append((train_dataloader,test_dataloader))
    return data_loaders

        
    


In [10]:
# tensor_data=rgb_data
print(all_data[1][0].shape)
print(all_data[1][1].shape)


torch.Size([234, 3, 201, 192])
torch.Size([234])


In [11]:
from torch.utils.data import random_split





In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
device = torch.device("cpu")

cuda


In [13]:
class MyCNN(nn.Module):
    def __init__(self, num_classes,data):
        super(MyCNN, self).__init__()
        self.data = data
        
        self.criterion = nn.CrossEntropyLoss().to(device)
        self.resnet = resnet50(pretrained=True)
        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(num_features, num_classes)
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
    
    def train(self,num_epochs):
        super()
        train_dataloader = self.data[0]
        for epoch in range(num_epochs):
            for batch in train_dataloader:
                cumulative_accuracy = torch.tensor([]).to(device)
                inputs, labels = batch
                inputs = inputs.to(device)
                labels= labels.to(device)
                self.optimizer.zero_grad()
                outputs = self(inputs).to(device)  # Add a channel dimension to the input
                loss = self.criterion(outputs, labels).to(device)
                loss.backward()
                self.optimizer.step()
                train_predictions = torch.argmax(outputs, dim=1).to(device)
                train_acc = (train_predictions == labels).float().to(device)
                cumulative_accuracy = torch.cat([cumulative_accuracy,train_acc], dim=0).to(device)
            print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {loss.item():.4f}, Train Accuracy: {cumulative_accuracy.mean().item():.4f}")
            self.eval(epoch,num_epochs)
           
    def eval(self,epoch,num_epochs):
        super()
        test_dataloader = self.data[1]
        with torch.no_grad():
            cumulative_accuracy = torch.tensor([]).to(device)
            for batch in test_dataloader:
                inputs, labels = batch
                inputs = inputs.to(device)
                labels = labels.to(device)
                val_outputs = self.forward(inputs).to(device)
                val_loss = self.criterion(val_outputs, labels).to(device)
                val_predictions = torch.argmax(val_outputs, dim=1).to(device)
                val_accuracy = (val_predictions == labels).float().to(device)
                cumulative_accuracy = torch.cat([cumulative_accuracy,val_accuracy], dim=0).to(device)
            print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss.item():.4f}, Validation Accuracy: {cumulative_accuracy.mean().item():.4f}")
    def forward(self, x):
        x = self.resnet(x)
        return x

In [29]:
# Step 4: Model Compilation

def create_model_list():
    model_list = []
    data_loaders = create_data_loaders()
    for i in range(31):
        model = MyCNN(2,data_loaders[i])
        model_list.append(model)
        model.to(device)
    return model_list
    


    
    

In [15]:
num_epochs = 1
batch_size = 32

In [30]:

for i in range(5):
    counter = 0
    print("Exepriment " , i)
    model_list = create_model_list()
    for model in model_list:
        print("model: ",counter)
        model.train(num_epochs)
        counter+=1

    

Exepriment  0


C:\Users\eden9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\eden9\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


model:  0
Epoch 1/1, Train Loss: 0.4130, Train Accuracy: 0.9091
Epoch 1/1, Validation Loss: 0.2831, Validation Accuracy: 0.8936
model:  1
Epoch 1/1, Train Loss: 0.0008, Train Accuracy: 1.0000
Epoch 1/1, Validation Loss: 0.0002, Validation Accuracy: 0.9787
model:  2
Epoch 1/1, Train Loss: 0.0021, Train Accuracy: 1.0000
Epoch 1/1, Validation Loss: 0.0220, Validation Accuracy: 0.9787
model:  3
Epoch 1/1, Train Loss: 0.0001, Train Accuracy: 1.0000
Epoch 1/1, Validation Loss: 0.5981, Validation Accuracy: 0.9574
model:  4
Epoch 1/1, Train Loss: 0.0140, Train Accuracy: 1.0000
Epoch 1/1, Validation Loss: 0.1781, Validation Accuracy: 0.9787
model:  5
Epoch 1/1, Train Loss: 0.0005, Train Accuracy: 1.0000
Epoch 1/1, Validation Loss: 0.0003, Validation Accuracy: 0.9787
model:  6
Epoch 1/1, Train Loss: 0.0301, Train Accuracy: 1.0000
Epoch 1/1, Validation Loss: 0.0333, Validation Accuracy: 0.9574
model:  7
Epoch 1/1, Train Loss: 1.4832, Train Accuracy: 0.8182
Epoch 1/1, Validation Loss: 0.3288, Vali